In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
sns.set()

In [21]:
df = pd.read_csv('Default.csv')

In [22]:
df.head()

,default,student,balance,income
0,No,No,729.526495,44361.62507
1,No,Yes,817.180407,12106.13470
2,No,No,1073.549164,31767.13895
3,No,No,529.250605,35704.49394
4,No,No,785.655883,38463.49588


In [23]:
df= pd.get_dummies(df, drop_first=True)

In [25]:
df = df[['default_Yes','balance', 'income',  'student_Yes']]
df.head()

,default_Yes,balance,income,student_Yes
0,0,729.526495,44361.62507,0
1,0,817.180407,12106.13470,1
2,0,1073.549164,31767.13895,0
3,0,529.250605,35704.49394,0
4,0,785.655883,38463.49588,0


In [39]:
df.columns = ['Default', 'balance', 'income', 'student_Yes']

In [40]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Default,10000.0,0.033300,0.179428,0.000000,0.000000,0.000000,0.000000,1.000000
balance,10000.0,835.374886,483.714985,0.000000,481.731105,823.636973,1166.308387,2654.322576
income,10000.0,33516.981876,13336.639563,771.967729,21340.462905,34552.644800,43807.729275,73554.233500
student_Yes,10000.0,0.294400,0.455795,0.000000,0.000000,0.000000,1.000000,1.000000


In [64]:
y = df['Default']
X = df.drop('Default', axis=1) # becareful inplace= False

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=200)

In [65]:
X_train.head()

,balance,income,student_Yes
5541,955.343370,40368.59789,0
5408,500.155117,34437.70689,0
2364,1546.760526,32928.63613,0
1257,1172.300890,34513.61085,0
6342,601.838501,37697.37005,0


In [66]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
balance,8000.0,839.823337,485.047819,0.000000,482.490231,828.873650,1173.486803,2578.469022
income,8000.0,33472.681344,13399.826760,771.967729,21247.819863,34492.198105,43694.845043,73554.233500
student_Yes,8000.0,0.297375,0.457132,0.000000,0.000000,0.000000,1.000000,1.000000


Scaling the X

In [67]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test) 

In [68]:
(955.34-839.82)/485.047

0.238162487346587

In [70]:
X_train_sc

array([[ 0.23817704,  0.51465946, -0.6505646 ],
       [-0.70032157,  0.07202226, -0.6505646 ],
       [ 1.45754985, -0.04060345, -0.6505646 ],
       ...,
       [-1.73153202, -1.17033067,  1.53712636],
       [-1.63068647,  0.12556851, -0.6505646 ],
       [-0.0645664 ,  0.07988195, -0.6505646 ]])

In [72]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(solver='lbfgs')
logistic.fit(X_train_sc, y_train.values.reshape(-1,))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [73]:
logistic.coef_

array([[ 2.72908852,  0.05897109, -0.24536815]])

In [74]:
y_pred_test = logistic.predict(X_test_sc)

In [75]:
# Fitting logistic regression to the training set using Statsmodel. Unscaled data! 

logit = sm.Logit(y_train, X_train)
logit.fit().summary()

Optimization terminated successfully.
         Current function value: inf
         Iterations 9


C:\Users\Pedram\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1789: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
C:\Users\Pedram\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\Pedram\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Default   No. Observations:                 8000
Model:                          Logit   Df Residuals:                     7997
Method:                           MLE   Df Model:                            2
Date:                Thu, 03 Oct 2019   Pseudo R-squ.:                     inf
Time:                        11:22:15   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
balance         0.0028      0.000     20.421      0.000       0.003       0.003
income         -0.0002   5.61e-06    -30.713      0.000      -0.000      -0.000
student_Yes    -3.6971      0.165    -22.369      0.000      -4.021      -3.373
===============================================================================
"""

### the Confusion Matrix 

In [76]:
from sklearn.metrics import confusion_matrix

In [77]:
print(confusion_matrix(y_test, y_pred_test))

[[1943    8]
 [  30   19]]


In [78]:
# defining our own confusion matrix function
from sklearn.metrics import confusion_matrix
def my_confusion_matrix(y, y_hat):
    cm = confusion_matrix(y, y_hat)
    TN, FP, FN, TP = cm[0,0], cm[0,1], cm[1,0], cm[1,1]
    accuracy = round((TP+TN) / (TP+ FP+ FN+ TN) ,2)
    precision = round( TP / (TP+FP),2)
    recall = round( TP / (TP+FN),2)
    cm_labled = pd.DataFrame(cm, index=['Actual : 0 ','Actual : 1'], columns=['Predict : 0','Predict :1 '])
    print('\n')
    print('Accuracy = {}'.format(accuracy))
    print('Precision = {}'.format(precision))
    print('Recall = {}'.format(recall))
    print("-----------------------------------------")
    return cm_labled
 

In [79]:
# Accuracy: what percentage of the total dots did we predict correctly?
# precision: among the ones that we predicted red, how many are actually red (incorporating FP: type I error)
# recall: how many of the red dots we predicted correctly (incorporating FN: type II error)
my_confusion_matrix(y_test,y_pred_test)



Accuracy = 0.98
Precision = 0.7
Recall = 0.39
-----------------------------------------


,Predict : 0,Predict :1
Actual : 0,1943,8
Actual : 1,30,19


###  Cross validation

Now let's try to get an estimate for the accuracy of our model using cross validation technique. 

In [80]:
from sklearn.model_selection import cross_val_score

In [81]:
# remember we saved our model in an object called "logistic"
accuracy = cross_val_score(estimator = logistic, X = X_train_sc, y = y_train, cv = 10 , scoring="accuracy" )

# How do you fix the warning? 

In [82]:
accuracy

array([0.96878901, 0.96754057, 0.97503121, 0.97128589, 0.97125   ,
       0.97125   , 0.97246558, 0.97121402, 0.97371715, 0.96996245])

In [83]:
round(accuracy.mean(),2)

0.97

In [85]:
round(accuracy.std(),4)

0.0021

## More reports

In [86]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, precision_recall_fscore_support, precision_recall_curve

In [88]:
# for example, let's take a look at classification_report
# for more information, read additional links number 2, and 3.
print(classification_report(y_test,y_pred_test))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1951
           1       0.70      0.39      0.50        49

    accuracy                           0.98      2000
   macro avg       0.84      0.69      0.75      2000
weighted avg       0.98      0.98      0.98      2000



Now let's look at some numbers for unscaled data!

In [89]:
logistic_usc = LogisticRegression(solver='lbfgs')
logistic_usc.fit(X_train, y_train.values.reshape(-1,))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [93]:
y_pred_test_usc = logistic_usc.predict(X_test)

In [90]:
logistic.coef_

array([[ 2.72908852,  0.05897109, -0.24536815]])

In [92]:
logistic_usc.coef_

array([[ 4.04919745e-03, -1.32557777e-04, -3.79463809e+00]])

In [94]:
my_confusion_matrix(y_test,y_pred_test)



Accuracy = 0.98
Precision = 0.7
Recall = 0.39
-----------------------------------------


,Predict : 0,Predict :1
Actual : 0,1943,8
Actual : 1,30,19


In [95]:
my_confusion_matrix(y_test,y_pred_test_usc)



Accuracy = 0.98
Precision = 0.5
Recall = 0.22
-----------------------------------------


,Predict : 0,Predict :1
Actual : 0,1940,11
Actual : 1,38,11


Remember the the output of Statsmodel and sklearn are not exactly equal! 
You can find the reason here: https://stats.stackexchange.com/questions/203740/logistic-regression-scikit-learn-vs-statsmodels

###  Done!